In [1]:
#importing the necessary files
import sys
sys.path.insert(1, '/Users/quintengeerts/Desktop/Algos/twitter_analyser/twitter_analyser')

In [2]:
import Tweet
import TwitterUser

In [3]:
#get the scraped twitterusers from the database
UserDA = TwitterUser.TwitterUserDA()
users = UserDA.load_all()

In [4]:
print(users)


Twitter user: name = traderstewie, id = 52166809
Twitter user: name = jmoneystonks, id = 1281015843044970496
Twitter user: name = Thrackx, id = 40942576
Twitter user: name = thetradejourney, id = 1208632009817354241
Twitter user: name = TraderAmogh, id = 3270306985
Twitter user: name = JackDamn, id = 58808528
Twitter user: name = Scelliott81, id = 1611593304
Twitter user: name = TradeWithNinja, id = 814893508066680832
Twitter user: name = CitronResearch, id = 236953420
Twitter user: name = MartyChargin, id = 58652546
Twitter user: name = AdamMancini, id = 39117589
Twitter user: name = HiddenPivots, id = 295299583
Twitter user: name = Pharmdca, id = 1884013189
Twitter user: name = Trader182, id = 1196947949013684224
Twitter user: name = ChartingOptions, id = 1302069535147687936
Twitter user: name = 87AlwaysRed, id = 930177090971877377
Twitter user: name = AshwinSamant, id = 427170268
Twitter user: name = AvatarAidan, id = 3083109892
Twitter user: name = gvstrader, id = 1034036822081499

In [16]:
#get scraped tweets from the db
TweetDA = Tweet.TweetDA()
tweets = Tweet.TweetList('stonks')
for usr in users:
    twt = TweetDA.load_all(username='@'+usr.username)
    tweets.add_all_tweets(twt)

In [6]:
len(tweets.tweets)

40632

In [15]:
#tweet looks like this
print(tweets[10250])

Traderstewie: @traderstewie * 2013-02-01T20:52:03.000Z
One of my top LONG ideas going into next week. Worth eyeing this setup in $C . Momentum play http://stks.co/cJkm 
--------------------------------------------------------------------------


In [90]:
#Sentiment analysis is only usefull if the tweet contains a ticker so lets sort the tweets only for those
def contains_ticker(txt):
    #split the sentence
    for word in txt.split():
        #check if it contains a ticker
        if word.startswith("$") and word.replace('$','').isalpha():
            return True
    return False

#noticed that a lot of replies to other peoples trade suggestion register as super bullish so i'll remove them 
def is_not_a_reply(txt):
    for word in txt.split():
        if word.startswith("@"):
            return False
    return True

In [91]:
filtered_tweets = Tweet.TweetList('filtered')
#loop over the tweets to only keep the ones that contain a ticker
for twt in tweets:
    if contains_ticker(str(twt.text)) and is_not_a_reply(str(twt.text)):
        filtered_tweets.add_tweet(twt)
len(filtered_tweets.tweets)

10426

In [92]:
import pandas as pd
import tqdm
#initialise dataframe
cols = ['username', 'screen_name','user_id', 'tweet_id', 'timestamp', 'text', 'likes', 'retweets', 
        'comments', 'emojis', 'image_link','is_reply', 'is_reply_to','URL', 'label']
tweet_df = pd.DataFrame(columns=cols)
for twt in tqdm.tqdm(filtered_tweets):
    usrn = twt.username
    scrn = twt.screen_name
    usrid = twt.user_id
    twtid = twt.tweet_id
    time = twt.timestamp
    txt = twt.text
    like = twt.likes
    rtwts = twt.retweets
    comts = twt.comments
    emji = twt.emojis
    img = twt.image_link
    isrpl = twt.is_reply
    to = twt.is_reply_to
    url = twt.URL
    #create the row
    row = pd.DataFrame([[usrn, scrn, usrid, twtid, time, txt, like, rtwts, comts, emji, img, isrpl, to, url, None]],
                      columns=cols)
    #append it to df
    tweet_df = tweet_df.append(row)

10426it [01:00, 172.68it/s]


In [93]:
#add an index so I can get rows like this
index = pd.Index(range(0,len(tweet_df)))
tweet_df = tweet_df.set_index(index)
tweet_df.tail()

,username,screen_name,user_id,tweet_id,timestamp,text,likes,retweets,comments,emojis,image_link,is_reply,is_reply_to,URL,label
10421,@yatesinvesting,Yates Investing,1118235493030866944,94bb3d88e501ae50d2029055c144e9c96e2289d3,2021-02-04T13:37:45.000Z,Adding $btbt here,2,57,8,None,[],0,,https://twitter.com/yatesinvesting/status/1357...,None
10422,@yatesinvesting,Yates Investing,1118235493030866944,4f3c01fb5cc76440fa8b1e7feb370e26fdc54b60,2021-02-04T18:18:50.000Z,$ENZC,44,240,27,🔥,[],0,,https://twitter.com/yatesinvesting/status/1357...,None
10423,@yatesinvesting,Yates Investing,1118235493030866944,f3052070a5722f99607f7b23742c27e218b81906,2021-02-03T14:44:17.000Z,$cciv dips pay,20,288,15,None,[],0,,https://twitter.com/yatesinvesting/status/1356...,None
10424,@yatesinvesting,Yates Investing,1118235493030866944,0f4a506ca63e5739922fb384590d4ae22eb1e05a,2021-02-04T20:36:38.000Z,$AACG WOW,16,100,19,None,[],0,,https://twitter.com/yatesinvesting/status/1357...,None
10425,@yatesinvesting,Yates Investing,1118235493030866944,2e7a853023ad5f45477765d8b84a33dd9b4fde93,2021-02-03T22:21:44.000Z,$KMPH multi bagger coming.,55,301,32,None,[],0,,https://twitter.com/yatesinvesting/status/1357...,None


---
I'll use vader NLP package to begin with, even though it probably wont be a very good model for sentiment analysis on this data

In [70]:
#import the NLP package and download model
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/quintengeerts/nltk_data...


True

In [94]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
#run the sentiment analysis on the data
results = list()
for txt in tqdm.tqdm(tweet_df['text']):
    score = SIA().polarity_scores(txt)
    score['txt'] = txt
    results.append(score)

100%|██████████| 10426/10426 [01:08<00:00, 152.53it/s]


In [100]:
#only use the sufficiently bullish and bearish tweets
res_filtered = list()
for i in results:
    if abs(i['compound']) > 0.8:
        res_filtered.append(i)
len(res_filtered)

461